Explore different trajectory parameterization options.

Step 1 is how to parameterize a continuous 2D $(x(t),y(t))$ trajectory. 
We can use a polynomial, a spline, or a neural network.

Step 2 is to use differential flatness to obtain the full state and control trajectories.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.graph_objects as go

# autoreload
%load_ext autoreload
%autoreload 2

# Step 1: Trajectory parameterization
---

## Polynomial

In [ ]:
class PolyPath:
    def __init__(self, d, n, start, end, free_coeffs=None):
        """
        Constrain endpoints, zero velocity at endpoints.

        d : int
            Dimension of the path
        n : int
            Degree of the polynomial
        free_coeffs : np.ndarray (n+1-4, d)
            Free coefficients. The first 4 coefficients are constrained
            
        """
        self.d = d
        self.n = n
        self.exponents = np.arange(n+1)
        
        # Compute full coefficients
        self.coeffs = np.zeros((n+1, d))
        self.coeffs[3:] = free_coeffs
        self.coeffs[3] = 2 * (start - end) - free_coeffs.T @ np.arange(2, n-1)
        self.coeffs[2] = (end - start) - self.coeffs[3] - free_coeffs.T @ np.arange(1, n-2)
        self.coeffs[1] = 0
        self.coeffs[0] = start
        

    def eval(self, t):
        N = len(t)
        T = t[:, None] ** self.exponents
        return T @ self.coeffs

In [ ]:
np.arange(2,5)

In [ ]:
np.ones((2,10)) @ np.ones(10)

In [ ]:
d = 2 # dim
n = 10 # degree
free_coeffs = np.random.randn(n+1-4, d)
start = np.array([0,0])
end = np.array([1,1])
path = PolyPath(d, n, start, end, free_coeffs)

t = np.linspace(0, 1, 100)
X = path.eval(t)
plt.plot(X[:, 0], X[:, 1])

In [ ]:
# 1-D
x_0 = 0.0
x_f = 1.0

x_list = []
a_5 = 1.0
a_4s = np.linspace(-10.0, 10.0, 10)

for a_4 in a_4s:
    a_3 = 2*((x_0 - x_f) - a_4)
    a_2 = (x_f - x_0) - a_3 - a_4
    a_1 = 0
    a_0 = x_0

    t = np.linspace(0.0, 1.0, 100)
    x = a_4*t**4 + a_3*t**3 + a_2*t**2 + a_1*t + a_0
    x_list.append(x)

In [ ]:
for x in x_list:
    plt.plot(t, x)

## B-spline

In [ ]:
import scipy.interpolate as spi

In [ ]:
start_point = np.array([0, 0])
end_point = np.array([1, 1])

control_points = np.array([
    [0.1, 0.1],
    [0.2, 0.5],
    [0.7, 0.6]
])

In [ ]:
points = np.vstack([start_point, control_points, end_point])
x = points[:, 0]
y = points[:, 1]

# Fit the B-spline
tck, u = spi.splprep([x, y], s=0)

# Evaluate the spline
unew = np.linspace(0, 1.0, 1000)
out = spi.splev(unew, tck)

# Plot the results
plt.figure()
plt.plot(x, y, 'ro', label='Control Points')
plt.plot(out[0], out[1], 'b-', label='B-spline Curve')
plt.legend()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('B-spline fitting with start and end points')
plt.show()

# Step 2: Differential Flatness
---

## Standard Dubin's car